<a href="https://colab.research.google.com/github/beyg1/Q4/blob/main/Openai%20SDK/Context_Handling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uq openai-agents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.6/130.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.4 MB/s eta 0:00:00


In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
from pydantic import BaseModel
from agents import (
    OpenAIChatCompletionsModel,
    RunConfig,
)
from openai import AsyncOpenAI
from google.colab import userdata

In [4]:
api = userdata.get('GOOGLE_API_KEY')
if not api:
   raise ValueError('GOOGLE_API_KEY not set')
external_client = AsyncOpenAI(
    api_key=api,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)

In [5]:
from agents import set_tracing_disabled
model = OpenAIChatCompletionsModel(
    openai_client = external_client,
    model = "gemini-2.0-flash",
)
set_tracing_disabled(True)

In [27]:
import asyncio
from dataclasses import dataclass
from agents import RunContextWrapper, Agent, Runner, function_tool

@dataclass
class User_base:
  name: str
  age: int

@function_tool
async def user_auth(wrapper : RunContextWrapper[User_base]) -> str:
  '''Returns the details of the User(i.e Name & Age of User)'''
  return f"User {wrapper.context.name} is {wrapper.context.age} years old."

async def main():
  user_info = User_base(name="Ali",age=20)

  agent = Agent(
      name = "User Authentication",
      tools = [user_auth],
      model = model,
  )

  result = await Runner.run(
      starting_agent = agent,
      input = "share current user's details",
      context = user_info,
      )
  print(result.final_output)

if __name__ == "__main__":
    asyncio.run(main())

OK. User Ali is 20 years old.

